## 1. Setup

In [15]:
import pandas as pd
from time import time
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import word_tokenize

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
dbase = '../assets/Preprocessed Data.xlsx'
data = pd.read_excel(dbase, sheet='Sheet1')

data.head()

,Author,Description,Long,Performance,Quality,Submission Date,Rating Date,One Year Date,Ticker,FDS Ticker,...,logPE,logPsales,logPFCF,logMKT,logEV,logROIC,MKT_category,Sentiment_polarity,Sentiment_subjectivity,Outperformed
1,Den1200,\nI recommend the purchase of BRK (and shortin...,True,3.9,2.6,2016-08-23,2016-09-06,2017-09-06,BRK.A,BRK.B,...,2.639003,0.489930,3.017299,12.813473,12.896157,1.999010,Large Cap,0.120324,0.429861,1
1,piggybanker,\n\nWe believe that the stock of 1-800-Flowers...,True,4.3,4.0,2013-10-16,2013-10-30,2014-10-30,FLWS,FLWS,...,3.228965,-0.793357,4.770579,4.949525,5.995618,2.256795,Small Cap,0.131239,0.502589,1
2,shoon1022,"\n \nAt $2.80, FLWS has an enterprise value of...",True,3.4,3.8,2011-02-20,2011-03-06,2012-03-05,FLWS,FLWS,...,0.000000,-1.316094,0.000000,4.330223,5.381191,0.000000,Small Cap,0.103768,0.380808,0
3,xds68,\n \n\n\n \n8/23/2017\n \nRecommending Seaboar...,True,0.0,0.0,2016-08-23,2016-09-06,2017-09-06,SEB,SEB,...,2.635309,-0.474458,2.675116,8.234805,8.042525,2.034737,Medium Cap,0.063402,0.422381,1
3,u0422811,\nThis pitch is pretty simple. Unfortunately...,False,5.2,4.9,2016-02-15,2016-02-29,2017-02-28,1PG,1PG-AU,...,0.000000,0.000000,0.000000,5.889505,5.603921,0.000000,Small Cap,0.121530,0.490904,0


In [4]:
data['Token description'] = data['Description'].apply(lambda x: word_tokenize(x))

In [5]:
descs = data['Token description'].map(lambda x: ' '.join(x))

In [6]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(descs)

print(dtm_tf.shape)

(6064, 15963)


In [9]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(descs)
print(dtm_tfidf.shape)

(6064, 15963)


In [11]:
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0,)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0,)
lda_tfidf.fit(dtm_tfidf)

c:\users\jrose\anaconda2\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
c:\users\jrose\anaconda2\lib\site-packages\sklearn\decomposition\online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [12]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
14     20.544714        1       1 -0.087671  0.020976
8      11.450827        1       2 -0.052989  0.125087
4       9.493683        1       3 -0.065050  0.040440
11      9.432624        1       4 -0.084811  0.101823
12      5.820287        1       5 -0.077193 -0.035343
3       5.204016        1       6 -0.067569  0.019975
7       4.806888        1       7 -0.034941 -0.006110
19      4.507804        1       8 -0.048015 -0.026226
1       4.181914        1       9 -0.101114 -0.048709
13      4.074452        1      10 -0.065141  0.068890
10      3.630678        1      11 -0.042499  0.040641
5       3.461041        1      12 -0.014822 -0.136594
0       3.179884        1      13 -0.060201 -0.105605
18      2.720962        1      14 -0.000046  0.229165
15      2.571256        1      15 -0.052087 -0.044526
17      2.343371        1      16  0.008788 -0.002424
2       1.006462        1      17  0.083774 -0.149529
16      0.895642        1      18  0.082343 -0.120418
9       0.478976        1      19  0.280830 -0.158139
6       0.194517        1      20  0.398412  0.186627, topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
8867   Default  25008.000000     million  25008.000000  30.0000  30.0000
9666   Default   7399.000000         oil   7399.000000  29.0000  29.0000
2687   Default   5988.000000         com   5988.000000  28.0000  28.0000
6108   Default   6295.000000         gas   6295.000000  27.0000  27.0000
10955  Default   8295.000000  production   8295.000000  26.0000  26.0000
13650  Default   5362.000000      stores   5362.000000  25.0000  25.0000
12813  Default   8642.000000    services   8642.000000  24.0000  24.0000
10960  Default  11264.000000    products  11264.000000  23.0000  23.0000
11367  Default   6686.000000       rates   6686.000000  22.0000  22.0000
1232   Default   5264.000000        bank   5264.000000  21.0000  21.0000
6850   Default   3945.000000        http   3945.000000  20.0000  20.0000
8270   Default   4290.000000       loans   4290.000000  19.0000  19.0000
13646  Default   4625.000000       store   4625.000000  18.0000  18.0000
942    Default  13271.000000      assets  13271.000000  17.0000  17.0000
15899  Default   3481.000000         www   3481.000000  16.0000  16.0000
3796   Default   6596.000000      demand   6596.000000  15.0000  15.0000
3510   Default   9404.000000   customers   9404.000000  14.0000  14.0000
2033   Default   5999.000000    capacity   5999.000000  13.0000  13.0000
1617   Default   6883.000000        book   6883.000000  12.0000  12.0000
8268   Default   4062.000000        loan   4062.000000  11.0000  11.0000
10954  Default   9006.000000     product   9006.000000  10.0000  10.0000
12120  Default   5052.000000      retail   5052.000000   9.0000   9.0000
6253   Default   2678.000000        gold   2678.000000   8.0000   8.0000
4347   Default   6759.000000    dividend   6759.000000   7.0000   7.0000
13234  Default   3533.000000    software   3533.000000   6.0000   6.0000
6368   Default   6958.000000       gross   6958.000000   5.0000   5.0000
1085   Default   1823.000000        auto   1823.000000   4.0000   4.0000
3509   Default   6958.000000    customer   6958.000000   3.0000   3.0000
15238  Default   5504.000000        used   5504.000000   2.0000   2.0000
12172  Default  10143.000000    revenues  10143.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
1749   Topic20     29.412866     brewers     30.364992   6.2105  -5.7301
646    Topic20     29.342888    anheuser     30.308557   6.2100  -5.7325
6633   Topic20     26.354425    heineken     27.306600   6.2069  -5.8399
1750   Topic20     26.221630     brewing     27.174124   6.2067  -5.8450
15759  Topic20     26.068403       wines     27.020529   6.2065  -5.

In [16]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

TypeError: (2.9846204554629992e-06+0j) is not JSON serializable

PreparedData(topic_coordinates=            Freq  cluster  topics                       x  \
topic                                                       
15     92.745838        1       1     (0.165252249654+0j)   
14      0.386797        1       2   (-0.0097354478414+0j)   
13      0.386631        1       3  (-0.00996575674916+0j)   
19      0.381220        1       4  (-0.00856091407095+0j)   
7       0.381220        1       5  (-0.00856194199656+0j)   
1       0.381220        1       6   (-0.0085618932383+0j)   
2       0.381220        1       7  (-0.00856176775671+0j)   
3       0.381220        1       8  (-0.00856197924159+0j)   
4       0.381220        1       9  (-0.00856197587869+0j)   
5       0.381220        1      10  (-0.00856188472784+0j)   
6       0.381220        1      11  (-0.00856196428585+0j)   
9       0.381220        1      12  (-0.00856187695722+0j)   
8       0.381220        1      13  (-0.00856195506603+0j)   
18      0.381220        1      14  (-0.00856196368144+